In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
texts = pd.read_csv("texts.csv")
texts.index= texts["id"]

for index, row in texts.iterrows():
    texts.at[index, "text"]=row["text"].lower()
    
del texts["id"]
train_df = pd.read_csv("train.csv")

texts.head()

train = train_df.copy(deep=True)
train = train.astype({'Text A':'string','Text B':'string'})

In [2]:
for index, row in train_df.iterrows():
    train.at[index, "Text A"]= texts.at[train_df.at[index, "Text A"], 'text']
    train.at[index, "Text B"]= texts.at[train_df.at[index, "Text B"], 'text']
train.head()

,Text A,Text B,Result
0,"a long, long time ago there lived in a little ...",her husband gasped at the audacity of the idea...,0
1,"a long, long time ago there lived in a little ...","she ate her supper with a hearty appetite, sai...",1
2,"a long, long time ago there lived in a little ...","now it came to pass on the third day, that est...",1
3,"a long, long time ago there lived in a little ...",the peasant promised to buy his daughters what...,1
4,"a long, long time ago there lived in a little ...",diplomacy is the practice of conducting negoti...,1


In [3]:

# Count vectorization of text
from sklearn.feature_extraction.text import CountVectorizer

corpus1 = train['Text A'].values
corpus2 = train['Text B'].values 
corpus = texts["text"].values
# Creating the vectorizer
vectorizer=feature_extraction.text.TfidfVectorizer(max_features=500, ngram_range=(1,8))
#vectorizer = CountVectorizer(max_features=500)
vectorizer.fit(corpus) 
# Converting the text to numeric data
X1 = vectorizer.transform(corpus1)
X2 = vectorizer.transform(corpus2) 
#print(vectorizer.get_feature_names())
 
# Preparing Data frame For machine learning
# Priority column acts as a target variable and other columns as predictors
CountVectorizedData1=pd.DataFrame(X1.toarray(), columns=vectorizer.get_feature_names())
#CountVectorizedData1.insert(0, "textinit",train['Text A'].values)
CountVectorizedData1 = CountVectorizedData1.add_suffix('_1')
print(CountVectorizedData1.shape)
CountVectorizedData2=pd.DataFrame(X2.toarray(), columns=vectorizer.get_feature_names())
#CountVectorizedData2.insert(0, "textinit",train['Text B'].values)
CountVectorizedData2 = CountVectorizedData2.add_suffix('_2')
print(CountVectorizedData2.shape)
X_train=pd.concat([CountVectorizedData1,CountVectorizedData2], axis=1)
y_train=train["Result"].values

(38939, 500)
(38939, 500)


In [4]:
# Count vectorization of text
from sklearn.feature_extraction.text import CountVectorizer
validation_df = pd.read_csv("validation.csv")
validation = validation_df.copy(deep=True)
validation = validation.astype({'Text A':'string','Text B':'string'})

for index, row in validation_df.iterrows():
    validation.at[index, "Text A"]= texts.at[validation_df.at[index, "Text A"], 'text']
    validation.at[index, "Text B"]= texts.at[validation_df.at[index, "Text B"], 'text']

corpus1 = validation['Text A'].values
corpus2 = validation['Text B'].values 
# Creating the vectorizer
#vectorizer = CountVectorizer(max_features=500)
#vectorizer=feature_extraction.text.TfidfVectorizer(max_features=500, ngram_range=(1,6))
# Converting the text to numeric data
X1 = vectorizer.transform(corpus1)
X2 = vectorizer.transform(corpus2) 
#print(vectorizer.get_feature_names())
 
# Preparing Data frame For machine learning
# Priority column acts as a target variable and other columns as predictors
CountVectorizedData1=pd.DataFrame(X1.toarray(), columns=vectorizer.get_feature_names())
#CountVectorizedData1.insert(0, "textinit",validation['Text A'].values)
CountVectorizedData1 = CountVectorizedData1.add_suffix('_1')
print(CountVectorizedData1.shape)
CountVectorizedData2=pd.DataFrame(X2.toarray(), columns=vectorizer.get_feature_names())
#CountVectorizedData2.insert(0, "textinit",validation['Text B'].values)
CountVectorizedData2 = CountVectorizedData2.add_suffix('_2')
print(CountVectorizedData2.shape)
X_validation=pd.concat([CountVectorizedData1,CountVectorizedData2], axis=1)
#X_validation = X_validation.drop([text])
y_validation=validation["Result"].values


(4194, 500)
(4194, 500)


In [5]:
from sklearn.svm import SVC
from sklearn.metrics import  plot_confusion_matrix, balanced_accuracy_score, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=50, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=200, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=20)

base_preds=model.predict(X_validation)
base_preds = [0 if val < 0.5 else 1 for val in base_preds]
m1=accuracy_score(y_validation, base_preds)
print(m1)

Epoch 1/10
1947/1947 [==============================] - 2s 804us/step - loss: 0.6224 - accuracy: 0.6514
Epoch 2/10
1947/1947 [==============================] - 2s 839us/step - loss: 0.5945 - accuracy: 0.6802
Epoch 3/10
1947/1947 [==============================] - 2s 855us/step - loss: 0.5662 - accuracy: 0.7043
Epoch 4/10
1947/1947 [==============================] - 2s 812us/step - loss: 0.5228 - accuracy: 0.7392
Epoch 5/10
1947/1947 [==============================] - 2s 792us/step - loss: 0.4742 - accuracy: 0.7718
Epoch 6/10
1947/1947 [==============================] - 2s 795us/step - loss: 0.4231 - accuracy: 0.8007
Epoch 7/10
1947/1947 [==============================] - 2s 809us/step - loss: 0.3722 - accuracy: 0.8216
Epoch 8/10
1947/1947 [==============================] - 2s 782us/step - loss: 0.3084 - accuracy: 0.8389
Epoch 9/10
1947/1947 [==============================] - 2s 794us/step - loss: 0.1476 - accuracy: 0.8397
Epoch 10/10
132/132 [==============================] - 0s 550us/